## Named entity recognition with spacy

In [32]:
import spacy
import pandas as pd
import numpy as np

nlp = spacy.load('en_core_web_sm')

In [33]:
# Import the email modules we'll need
import glob
import email
import mailparser
from email import policy
from email.parser import BytesParser

path = '../datawe/raw/Email_Classification/*'
email_types = glob.glob(path)
appendFilesData = []
file_raw_data = [] 
for folder in email_types:
    files = glob.glob(folder+"/*.txt")
    email_type = folder.split('\\')[1]
    for name in files:
        try:
            with open(name) as fp:
                raw_data = fp.read()
                file_raw_data.append(raw_data)
                msg = mailparser.parse_from_string(raw_data)
                appendFilesData.append({
                    "to":msg.to,
                    "from":msg.from_,
                    "subject":msg.subject,
                    "date":msg.date,
#                     "sent":msg["Sent"],
#                     "importance":msg["Importance"],
                    "content":  msg.body,
                    "class_to_exec":email_type,
                })
         
        except IOError as exc:
            print('Exception')

            
#creating pandas dataframe
data = pd.DataFrame(appendFilesData)

In [34]:
data

,class_to_exec,content,date,from,subject,to
0,Booking,"Dear Rohit,\n \nAs per our conversation a whil...",None,"[(, Meemendra), (, ), (, meemendra.kumar@andbe...",Mr. Nagdev X 04,"[(, Gateway)]"
1,Booking,"Hi Agnes,\n \nI have not received a response t...",None,"[(, Kurien), (, Vinay), (, ), (, vkurien@jones...",RE: Rate Proposal - Taj Connemara,"[(Reservations, Reservations@tajhotels.com)]"
2,Booking,"As per corporate holiday plan, kindly arrange ...",None,"[(, tehp@tatasteel.com[mailto:tehp@tatasteel.c...",TEHP/18/12809: Booking request of Arun Gadamsh...,"[(Reservations Coorg, reservations.coorg@tajho..."
3,Booking,"Dear Gampa ji,\n \nGreetings of the day !\n \n...",None,"[(, INDIAN), (, ), (, indianjourney@gmail.com)]",Valley Period/3 double room/Lerner group,"[(, Neeraj)]"
4,Cancellation,Hi\n\nPlease cancel my booking .,None,"[(, rajivbhatia@ase.ae[mailto:rajivbhatia@ase....",Re: Taj Chandigarh Payment link for reservatio...,"[(Reservations, Reservations@tajhotels.com)]"
5,Cancellation,"Dear Mr. Athalye,\n \nAs per your request, I h...",None,"[(, Reservations)]",RE: Taj Chandigarh Payment link for reservatio...,"[(, uday)]"
6,Cancellation,"Dear Team,\n \nPlease cancel the attached book...",None,"[(, Traveldesk), (, Star), (, ), (, traveldesk...",URGENT CANCELLATION for Mr. Ansil Antony @ Viv...,"[(, Corporate), (, ), (, Reservations), (, ), ..."
7,Enquiry,"Dear Team Inner Circle,\nI plan to stay at the...",None,"[(, sumitdasgupta70@gmail.com)]","A request for stay at Taj Mahal, Mumbai: Nov 30","[(, innercircle@tajhotels.com)]"
8,Enquiry,"Hi,\n \nYour hotel was recommended to me by th...",None,"[(, Maarten), (, ), (, maarten@freightquip.com)]",FW: ABB India Visit Info,"[(Reservations, Reservations@tajhotels.com)]"
9,Enquiry,"Dear Reservation Center,\nGood Day,\n \nI woul...",None,"[(, AMRO), (, ), (, aalsabban@saudiags.com)]",Airline Staff Rate,"[(Reservations, Reservations@tajhotels.com)]"


In [67]:
body = data["content"][:1]
entityData = []
for cnt in body:
    entityCh = []
    entity = nlp(cnt)
    for token in entity.ents:
        entityData.append({"text":token.text, "label":token.label_})
#     entityData.append(entityCh)


In [68]:
entityDF = pd.DataFrame(entityData)

In [70]:
entityDF["label"].value_counts()

TIME        1
PERSON      1
CARDINAL    1
DATE        1
ORG         1
Name: label, dtype: int64

In [75]:
type(entityDF["text"][1])

str

In [61]:
text = nlp('. '.join(np.array(data["content"])))

In [62]:
for token in text:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
          token.shape_, token.is_alpha, token.is_stop)

Dear dear PROPN NNP compound Xxxx True False
Rohit rohit PROPN NNP ROOT Xxxxx True False
, , PUNCT , punct , False False

 
 
 
 SPACE _SP  
 
 False False
As as ADP IN prep Xx True False
per per ADP IN prep xxx True True
our -PRON- ADJ PRP$ poss xxx True True
conversation conversation NOUN NN pobj xxxx True False
a a DET DT det x True True
while while NOUN NN npadvmod xxxx True True
ago ago ADV RB advmod xxx True False
, , PUNCT , punct , False False
please please INTJ UH intj xxxx True True
book book NOUN NN ROOT xxxx True False
and and CCONJ CC cc xxx True True
confirm confirm VERB VB conj xxxx True False
02 02 NUM CD nummod dd False False
Executive executive ADJ JJ compound Xxxxx True False
room room NOUN NN dobj xxxx True False
at at ADP IN prep xx True True
Taj taj PROPN NNP compound Xxx True False
Gateway gateway PROPN NNP compound Xxxxx True False
Ganges ganges PROPN NNP pobj Xxxxx True False
from from ADP IN prep xxxx True True
24/26 24/26 NUM CD nummod dd/dd False False
Decem

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



True
guests guest NOUN NNS pobj xxxx True False
. . PUNCT . punct . False False


 

 SPACE _SP  

 False False
No no DET DT det Xx True False
bills bill NOUN NNS nsubjpass xxxx True False
will will VERB MD aux xxxx True True
be be VERB VB auxpass xx True True
entertained entertain VERB VBN ROOT xxxx True False
by by ADP IN agent xx True True
us -PRON- PRON PRP pobj xx True True
until until ADP IN mark xxxx True True
it -PRON- PRON PRP nsubj xx True True
bears bear VERB VBZ advcl xxxx True False
the the DET DT det xxx True True
guest guest NOUN NN compound xxxx True False
signature signature NOUN NN dobj xxxx True False
. . PUNCT . punct . False False


 

 SPACE _SP  

 False False
Kindly kindly ADV RB advmod Xxxxx True False
confirm confirm VERB VBP ROOT xxxx True False
the the DET DT det xxx True True
booking booking NOUN NN dobj xxxx True False
at at ADP IN prep xx True True
the the DET DT det xxx True True
earliest early ADJ JJS pobj xxxx True False
with with ADP IN prep xxxx True

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 True False
. . PUNCT . punct . False False

 
 SPACE   
 False False
I -PRON- PRON PRP nsubj X True False
want want VERB VBP ROOT xxxx True False
to to PART TO aux xx True True
check check VERB VB xcomp xxxx True False
room room NOUN NN compound xxxx True False
availability availability NOUN NN dobj xxxx True False
for for ADP IN prep xxx True True
Monday monday PROPN NNP pobj Xxxxx True False
26 26 NUM CD nummod dd False False
November november PROPN NNP appos Xxxxx True False
to to ADP IN prep xx True True
Thursday thursday PROPN NNP pobj Xxxxx True False
29 29 NUM CD nummod dd False False
November november PROPN NNP appos Xxxxx True False
, , PUNCT , punct , False False
for for ADP IN prep xxx True True
1 1 NUM CD nummod d False False
person person NOUN NN pobj xxxx True False
. . PUNCT . punct . False False

 
 SPACE   
 False False
I\u2019 i\u2019 PROPN NNP ROOT X\xdddd False False
m m NOUN NN nsubj x True False
also also ADV RB advmod xxxx True True
after after ADP IN prep xxxx 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



ADP IN aux xx True True
is be VERB VBZ advcl xx True True
in in ADP IN prep xx True True
Dubai dubai PROPN NNP pobj Xxxxx True False
.. .. PUNCT . punct .. False False
Dear dear ADJ JJ compound Xxxx True False
Gaurav gaurav NOUN NN nsubj Xxxxx True False
, , PUNCT , punct , False False


 

 SPACE _SP  

 False False
Would would VERB MD aux Xxxxx True False
like like VERB VB ROOT xxxx True False
to to PART TO aux xx True True
know know VERB VB xcomp xxxx True False
a a DET DT det x True True
package package NOUN NN dobj xxxx True False
for for ADP IN prep xxx True True
2 2 NUM CD nummod d False False
nights night NOUN NNS pobj xxxx True False
in in ADP IN prep xx True True
Taj taj PROPN NNP compound Xxx True False
Agauda agauda PROPN NNP pobj Xxxxx True False
starting start VERB VBG advcl xxxx True False
17th 17th ADJ JJ amod ddxx False False
April april PROPN NNP npadvmod Xxxxx True False
2019 2019 NUM CD nummod dddd False False
to to ADP IN prep xx True True
19th 19th ADJ JJ amod ddx

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




nt not ADV RB advmod xx True False
the the DET DT det xxx True True
payment payment NOUN NN attr xxxx True False
due due ADJ JJ amod xxx True True
when when ADV WRB advmod xxxx True True
we -PRON- PRON PRP nsubj xx True True
check check VERB VBP advcl xxxx True False
in in PART RP prt xx True True
? ? PUNCT . punct ? False False


In [109]:
entityDtl = []
for token in text.ents:
    if len(token.text.rstrip()) > 0:
        entityDtl.append({"label":token.label_, "text":token.text})

In [110]:
entyDF = pd.DataFrame(entityDtl)
entyDF

,label,text
0,PERSON,Reservations\n
1,PERSON,Taj Dubai
2,MONEY,75677SB034081
3,CARDINAL,"1,245"
4,ORG,image002.png
5,CARDINAL,"4,124"
6,PERSON,Reservation
7,PERSON,75677SB034081\n\n
8,PERSON,Saroj Kumar\n\n
9,PERSON,Dates


In [106]:
type(entyDF["text"][3])

str

In [107]:
entyDF.drop(entyDF['text'] == '\n')

KeyError: '[False False False  True False False False  True  True False False False\n False False False False False False False  True False False False  True\n False  True False False  True  True  True  True False False False False\n False False False False False False False False False False False False\n False False False False False False False False False False False False\n False  True False  True  True  True False  True  True  True False  True\n  True False  True  True False  True False False  True  True False  True\n  True  True  True False] not found in axis'

In [92]:
text = nlp(u"""

   From: Reservations
     To: saroj_chanakya@yahoo.co.in
   Date: 11/19/2018 9:16:37 AM
Subject: Your Reservation at Taj Dubai is confirmed # 75677SB034081
Attachments: image001.png (1,245)
         image002.png (4,124)
-------------------------------------------------------------------
Dear Sir/ Madam,

 

Greetings of the day!

 

As per your request, I am pleased to assist you with the reservation
details as follows:

 

*         Reservation #: 75677SB034081

*         Guest Name: Saroj Kumar

*         Hotel: Taj Dubai

*         Dates of Stay: Tue Nov 20 - Sun Nov 25, 2018

*         Room: Luxury Burj View Room King Bed

*         Rate: Stay a Bit Longer – AED 6500.61

*         Inclusions: Breakfast, complimentary shuttle to Dubai mall at
fixed timings and Wi-Fi upto 4 devices 

*         Cancellation: Reservations must be cancelled by 2PM - 1 day
prior to arrival to avoid a penalty of 1 night charge plus taxes

 

Please do not hesitate to contact us if we may be of further assistance
to you.

 

Warm Regards,

Diana Chettiar

Taj Reservations Worldwide Team

 



 

THE INDIAN HOTELS COMPANY LIMITED

www.tajhotels.com
<https://apac01.safelinks.protection.outlook.com/?url=http%3A%2F%2Fwww.t
ajhotels.com%2F&data=02%7C01%7C%7Cd23e078bc62b4ec358be08d5c5f96f44%7C7e5
ee0d62a8a4e778e4b90c1cc745e1d%7C0%7C0%7C636632597448165689&sdata=%2BQU3X
8LKvzb54Fo71zFptCcoeVidIbKObxG1E8pWNIs%3D&reserved=0> 

Toll-free India: 1 800 111 825| Alternate network: +91 22 6601 1825 *

Email: reservations@tajhotels.com <mailto:reservations@tajhotels.com>  

USA & Canada: 1 866 969 1 825 | Bahrain Toll Free: 080004866 | Brazil
Toll Free: 08000381422

Egypt Toll Free: 08000000485 | United Arab Emirates (UAE) Toll Free:
800 035 702 467

Other countries: 00 800 4 588 1 825

*Local / STD charges as applicable**accessible 24 hours a day, 7 days a
week

WEBSITE
<https://apac01.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.
tajhotels.com%2Fen-in%2Fhome%2F&data=02%7C01%7C%7Ce8958d82c76b410d3dfc08
d538b41d0c%7C7e5ee0d62a8a4e778e4b90c1cc745e1d%7C0%7C0%7C6364772685935436
19&sdata=pEkmGdD62kvkOf853NOX60L%2Fg8B%2BF66bnutHLyEzTA0%3D&reserved=0>
| FACEBOOK
<https://apac01.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.
facebook.com%2FTajHotels&data=02%7C01%7C%7Ce8958d82c76b410d3dfc08d538b41
d0c%7C7e5ee0d62a8a4e778e4b90c1cc745e1d%7C0%7C0%7C636477268593543619&sdat
a=DPCJryD4ni6oOmIIe0V7ARRLKxon5MNn%2FfPfrbDOuIg%3D&reserved=0>  |
YOUTUBE
<https://apac01.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.
youtube.com%2Fuser%2FTajMovies&data=02%7C01%7C%7Ce8958d82c76b410d3dfc08d
538b41d0c%7C7e5ee0d62a8a4e778e4b90c1cc745e1d%7C0%7C0%7C63647726859354361
9&sdata=55s6ABV4Dj7Kz%2BfQhlJNa%2Fdp6SJBDjaCZKBs7nwzAJk%3D&reserved=0>
| TWITTER
<https://apac01.safelinks.protection.outlook.com/?url=https%3A%2F%2Ftwit
ter.com%2Ftajhotels&data=02%7C01%7C%7Ce8958d82c76b410d3dfc08d538b41d0c%7
C7e5ee0d62a8a4e778e4b90c1cc745e1d%7C0%7C0%7C636477268593543619&sdata=gwP
nLpEC1BQLP6QKE9yZkIYhLKN3btoC715F73p4qiI%3D&reserved=0> | INSTAGRAM
<https://apac01.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.
instagram.com%2Ftajhotels%2F&data=02%7C01%7C%7Ce8958d82c76b410d3dfc08d53
8b41d0c%7C7e5ee0d62a8a4e778e4b90c1cc745e1d%7C0%7C0%7C636477268593543619&
sdata=pkq6i0ulCXImb871jwhueC0X4Uoce5qnym7%2FKnhBpC8%3D&reserved=0> |
PINTEREST
<https://apac01.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.
pinterest.com%2Ftajhotels&data=02%7C01%7C%7Ce8958d82c76b410d3dfc08d538b4
1d0c%7C7e5ee0d62a8a4e778e4b90c1cc745e1d%7C0%7C0%7C636477268593543619&sda
ta=7XvZBgDqYQBH2bIWB88a8xvSflEq9Lcu6LXccxyrxh4%3D&reserved=0> 

 



 




 """)

In [60]:
text

'Dear Rohit,\n \nAs per our conversation a while ago, please book and confirm 02 Executive room at Taj Gateway Ganges from 24/26 December 2018 for two nights.. Hi Agnes,\n \nI have not received a response to this email as yet. Please could you let me know that the rooms have been confirmed?. As per corporate holiday plan, kindly arrange to book the following accommodation in your Hotel/Resort as per details given below:\nSerial number\n:\nTEHP/18/12809\nName\n:\nArun Gadamshetty\nP No.\n:\n153021\nDepartment\n:\nSpares Manufacturing Department\nContact No\n:\n8092084556\nPlace\n:\nVivanta by Taj - Madikeri Coorg, Coorg\nRoom Type\n:\nSuperior Charm\nCheck In Date-Time\n:\n10 Dec 2018 02:00 PM\nCheck Out Date-Time\n:\n12 Dec 2018 12:00 PM\nNo of rooms\n:\n1\nComments by requester\n:\nFor Honeymoon trip\n\n\nTraveller Details:\n1. \nName\n:\nArun Gadamshetty\n\nRelation\n:\nSelf\n\nAge\n:\n34 years\n2. \nName\n:\nSHWETA GADAMSHETTY\n\nRelation\n:\nSpouse\n\nAge\n:\n28 years\n\n\nAs agree

In [84]:
len("")

0

In [95]:
import spacy

nlp = spacy.load('en_core_web_sm')
doc = nlp(u'Apple is /n looking at buying U.K. startup for $1 billion')
for token in doc:
    print(token.text, token.pos_, token.dep_)

Apple PROPN nsubj
is VERB aux
/n PUNCT neg
looking VERB ROOT
at ADP prep
buying VERB pcomp
U.K. PROPN compound
startup NOUN dobj
for ADP prep
$ SYM quantmod
1 NUM compound
billion NUM pobj
